Update: 01.02.2025 - Unsure of what is contained in this notebook or my objective here. Although I think I was trying to find some features within the (single) head of GPT2 to integrate with other transformers. Think I was trying to scale the embeddings using softmax-like or trig (1-arcos(x)) equation but unclear. 

This notebook is here to serve as a memorial to my poor compute struggling with my terrible setup.

## 24.05.2024

In [ ]:
import tqdm as notebook_tqdm

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/Users/mimiphan/Library/Mobile Documents/com~apple~CloudDocs/Documents/MBA Documents/MyPlayground/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [73]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [77]:
output_params = {
    'output_scores': True,
    'output_logits': True,
    'output_attentions': True,
    'output_hidden_states': True,
    'return_dict_in_generate': True,
    'num_return_sequences': 1
}
model.generation_config


GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

In [78]:
model.generation_config.update(**output_params)

{}

In [79]:
model.generation_config

GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "output_attentions": true,
  "output_hidden_states": true,
  "output_logits": true,
  "output_scores": true,
  "return_dict_in_generate": true
}

In [80]:
inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
generation = model.generate(**inputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/mimiphan/Library/Mobile Documents/com~apple~CloudDocs/Documents/MBA Documents/MyPlayground/.conda/lib/python3.12/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [85]:
generation.__dict__.keys()

dict_keys(['sequences', 'scores', 'logits', 'attentions', 'hidden_states', 'past_key_values'])

In [88]:
len(generation["hidden_states"])

12

In [91]:
last_layer = torch.cat(generation["hidden_states"][-1])

In [92]:
last_layer.shape

torch.Size([13, 1, 768])

In [65]:
?GenerationConfig

Init signature: GenerationConfig(**kwargs)
Docstring:     
Class that holds a configuration for a generation task. A `generate` call supports the following generation methods
for text-decoder, text-to-text, speech-to-text, and vision-to-text models:

    - *greedy decoding* if `num_beams=1` and `do_sample=False`
    - *contrastive search* if `penalty_alpha>0.` and `top_k>1`
    - *multinomial sampling* if `num_beams=1` and `do_sample=True`
    - *beam-search decoding* if `num_beams>1` and `do_sample=False`
    - *beam-search multinomial sampling* if `num_beams>1` and `do_sample=True`
    - *diverse beam-search decoding* if `num_beams>1` and `num_beam_groups>1`
    - *constrained beam-search decoding* if `constraints!=None` or `force_words_ids!=None`
    - *assisted decoding* if `assistant_model` or `prompt_lookup_num_tokens` is passed to `.generate()`

To learn more about decoding strategies refer to the [text generation strategies guide](../generation_strategies).

<Tip>

A large numb

In [16]:
inputs.__dict__.keys()

dict_keys(['data', '_encodings', '_n_sequences'])

In [4]:
generation_output.__dict__.keys()

dict_keys(['sequences', 'scores', 'logits', 'attentions', 'hidden_states', 'past_key_values'])

In [11]:
inputs.data["input_ids"], generation.sequences

tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220]])

In [18]:
import torch

In [22]:
torch.cat(generation_output.scores).shape

torch.Size([12, 50257])

In [28]:
tokenizer.decode(generation_output.sequences.squeeze(0))

"Hello, my dog is cute and icky. I'm not sure if she's a good dog"

In [43]:
len(generation_output.past_key_values), len(generation_output.past_key_values[0]), len(generation_output.past_key_values[0][0])

(12, 2, 1)

## 25.05

In [34]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import tqdm as notebook_tqdm
import torch
import numpy as np

In [119]:
output_params = {
    'output_scores': True,
    'output_logits': False,
    'output_attentions': False,
    'output_hidden_states': False,
    'output_past_key_values': False,
}

off_params = {
    "max_new_tokens": 75,
    "num_return_sequences": 1
}

class KittyHouse:
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    model.generation_config.update(**output_params, **off_params)
    
    def __init__(self, user_input):
        tokens = KittyHouse.tokenizer(user_input, return_attention_mask=True, return_tensors="pt")
        output = KittyHouse.model.generate(**tokens, return_dict_in_generate=True)
        
        self.ids = tokens.data["input_ids"].squeeze(0) 
        self.mask = tokens.data["attention_mask"]
        # [num_input], [1, num_input]
        
        self.full_ids = output["sequences"].squeeze(0) # [1, num_input+num_gen]
        self.full_text = KittyHouse.tokenizer.decode(self.full_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        # [num_input+num_gen], [num_input+num_gen]
        
        self.pliers_ids = self.full_ids[self.ids.shape[-1]:]
        self.pliers_text = KittyHouse.tokenizer.decode(self.pliers_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        self.scores = torch.cat(output["scores"]) # note: only gives for the pliers
        self.emit = KittyHouse.model.compute_transition_scores(output["sequences"], output["scores"], normalize_logits=True)
        # [16, 50257] (== [max_new_tokens, vocab_size] == [num_gen, vocab_size]), [1, num_gen]
        print(self.full_text)

In [120]:
fish = KittyHouse("hey how are ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hey how are  you  going to get your hands on this?
I'm not sure if you're familiar with the term "candy" or "candy cake" but I'm sure you've heard of it. It's a type of cake that is made with a mixture of sugar and butter. It's made with a mixture of sugar and butter. It's made with a mixture of sugar and butter. It's made with a mixture of sugar and butter. It's made with a mixture
tensor([[-1.5353e+00, -1.1697e+00, -1.5543e+00, -6.6302e-03, -1.7125e+00,
         -4.1111e-01, -2.4712e+00, -2.7393e+00, -2.9382e+00, -1.7024e-01,
         -2.1970e+00, -2.0660e+00, -1.6528e+00, -2.2301e+00, -1.9165e+00,
         -1.6556e+00, -1.1757e+00, -1.6687e+00, -1.4346e+00, -2.0074e+00,
         -2.0726e+00, -2.2225e-02, -1.4035e+00, -2.6712e+00, -1.0819e+00,
         -4.6384e+00, -3.2495e+00, -2.3237e+00, -1.3640e+00, -1.4885e+00,
         -2.1724e+00, -1.7633e-01, -2.9107e+00, -1.4327e+00, -1.0871e+00,
         -1.4506e+00, -1.8902e+00, -1.3378e+00, -6.5544e-01, -1.7479e+00,
         -7.7603e-01, 

In [121]:
fish.__dict__.keys()

dict_keys(['ids', 'mask', 'full_ids', 'full_text', 'pliers_ids', 'pliers_text', 'scores', 'emit'])

In [123]:
fish.emit.shape

torch.Size([1, 100])

In [159]:
fish.emit.sigmoid().squeeze(0)

tensor([[0.1772, 0.2369, 0.1745, 0.4983, 0.1528, 0.3986, 0.0779, 0.0607, 0.0503,
         0.4575, 0.1000, 0.1124, 0.1607, 0.0971, 0.1282, 0.1604, 0.2358, 0.1586,
         0.1924, 0.1184, 0.1118, 0.4944, 0.1973, 0.0647, 0.2531, 0.0096, 0.0373,
         0.0892, 0.2036, 0.1842, 0.1023, 0.4560, 0.0516, 0.1927, 0.2522, 0.1899,
         0.1312, 0.2079, 0.3418, 0.1483, 0.3152, 0.3071, 0.2024, 0.3029, 0.0913,
         0.3908, 0.2601, 0.0384, 0.4967, 0.1562, 0.2171, 0.1894, 0.1805, 0.2109,
         0.1004, 0.0619, 0.4950, 0.1535, 0.3231, 0.1024, 0.2662, 0.1972, 0.3100,
         0.1042, 0.2987, 0.1661, 0.1657, 0.4935, 0.0712, 0.4120, 0.2865, 0.2735,
         0.2276, 0.3952, 0.3271, 0.4145, 0.2034, 0.3751, 0.4974, 0.2437, 0.4754,
         0.3716, 0.4494, 0.3384, 0.4686, 0.4698, 0.4703, 0.4021, 0.4585, 0.4995,
         0.4711, 0.4965, 0.4829, 0.4897, 0.3624, 0.4874, 0.4902, 0.4906, 0.4691,
         0.4822]])

In [158]:
1 - torch.exp(fish.emit)

tensor([[0.7846, 0.6895, 0.7887, 0.0066, 0.8196, 0.3371, 0.9155, 0.9354, 0.9470,
         0.1565, 0.8889, 0.8733, 0.8085, 0.8925, 0.8529, 0.8090, 0.6914, 0.8115,
         0.7618, 0.8657, 0.8741, 0.0220, 0.7543, 0.9308, 0.6610, 0.9903, 0.9612,
         0.9021, 0.7444, 0.7743, 0.8861, 0.1617, 0.9456, 0.7613, 0.6628, 0.7656,
         0.8490, 0.7376, 0.4808, 0.8259, 0.5398, 0.5568, 0.7462, 0.5654, 0.8995,
         0.3586, 0.6485, 0.9601, 0.0131, 0.8148, 0.7228, 0.7664, 0.7798, 0.7328,
         0.8884, 0.9341, 0.0199, 0.8186, 0.5226, 0.8859, 0.6372, 0.7544, 0.5507,
         0.8836, 0.5740, 0.8008, 0.8014, 0.0258, 0.9234, 0.2993, 0.5984, 0.6235,
         0.7054, 0.3465, 0.5139, 0.2922, 0.7447, 0.3996, 0.0102, 0.6778, 0.0936,
         0.4087, 0.1839, 0.4886, 0.1181, 0.1138, 0.1121, 0.3274, 0.1534, 0.0022,
         0.1092, 0.0138, 0.0660, 0.0404, 0.4315, 0.0491, 0.0383, 0.0370, 0.1166,
         0.0687]])